In [1]:
!pip install -r requirements.txt

In [2]:
%cd HoFpy

C:\Users\win\Spatial Relation\scene_labeling\HoFpy


In [3]:
!pip install . --user

Processing c:\users\win\spatial relation\scene_labeling\hofpy
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for hofpy: filename=hofpy-0.2.0-cp310-cp310-win_amd64.whl size=68425 sha256=a49f6674dfad65f7151ce04036a57d62912990433cda09e3063c188230060848
  Stored in directory: C:\Users\win\AppData\Local\Temp\pip-ephem-wheel-cache-ir9zf5u7\wheels\ba\9f\13\c8c2083b507d7dece9a88629db96cfe9e61ff30c279329614d
Successfully built hofpy
  Attempting uninstall: hofpy
    Found existing installation: hofpy 0.2.0
    Uninstalling hofpy-0.2.0:
      Successfully uninstalled hofpy-0.2.0


In [ ]:
""" 
This demo script runs the scene annotation system on the input images located in ./input/demo_images/
"""
from scene_labeling import SceneLabeling
from utils import draw_detection
import cv2
import os
import json

DEMO_IMAGES_DIR = './input/demo_images/'


def load_image_paths(img_dir=None):
    img_names = []
    img_paths = []
    for root, dirs, files in os.walk(img_dir):
        for f in files:
            img_path = img_dir + f
            img_paths.append(img_path)
            img_names.append(f)
    return img_names, img_paths


if __name__ == '__main__':
    model = SceneLabeling()
    image_names, image_paths = load_image_paths(DEMO_IMAGES_DIR)

    # Loop the image names in the demo dir and load them using OpenCV
    for name, path in zip(image_names, image_paths):
        image = cv2.imread(path, cv2.IMREAD_COLOR)

        model.process_image(image, name)
        annotation_results = model.get_image_annotations(name)

        print(f'{name} annotations:')
        for ar in annotation_results:
            print('Spatial relationship: ' + ar['spatial_relationship']
                  + '. General interaction: ' + ar['general_interaction'] + '. Person interaction: '
                  + ar['person_interaction'])
            arg_box = json.loads(ar['arg_bounding_box'])
            arg_label = ar['arg_label']
            ref_box = json.loads(ar['ref_bounding_box'])
            ref_label = ar['ref_label']
            draw_detection(image.copy(), [arg_box, ref_box], [arg_label, ref_label])


000000000036.jpg annotations:
Spatial relationship: person_1 overlaps and is below and to the right of umbrella_1. General interaction: Interacting. Person interaction: Carrying
000000000086.jpg annotations:
Spatial relationship: person_1 overlaps and is above motorbike_1. General interaction: Interacting. Person interaction: Riding


In [ ]:
### modified ##


"""
This demo script runs the scene annotation system on the input images located in ./input/demo_images/
"""
from scene_labeling import SceneLabeling
from utils import draw_detection
import cv2
import os
import json

DEMO_IMAGES_DIR = './input/demo_images/'


def load_image_paths(img_dir=None):
    img_names = []
    img_paths = []
    for root, dirs, files in os.walk(img_dir):
        for f in files:
            img_path = img_dir + f
            img_paths.append(img_path)
            img_names.append(f)
    return img_names, img_paths


if __name__ == '__main__':
    model = SceneLabeling()
    image_names, image_paths = load_image_paths(DEMO_IMAGES_DIR)

    # Loop the image names in the demo dir and load them using OpenCV
    for name, path in zip(image_names, image_paths):
        image = cv2.imread(path, cv2.IMREAD_COLOR)

        model.process_image(image, name)
        annotation_results = model.get_image_annotations(name)

        print(f'{name} annotations:')
        for ar in annotation_results:
            if ("person" not in ar['spatial_relationship']) and ("Interacting" in ar['general_interaction']):
                print('Spatial relationship: ' + ar['spatial_relationship']
                      + '. General interaction: ' + ar['general_interaction'] + '. Person interaction: '
                      + ar['person_interaction'])
                arg_box = json.loads(ar['arg_bounding_box'])
                arg_label = ar['arg_label']
                ref_box = json.loads(ar['ref_bounding_box'])
                ref_label = ar['ref_label']
                draw_detection(image.copy(), [arg_box, ref_box], [arg_label, ref_label])


In [5]:
import cv2
import os
import json
from scene_labeling import SceneLabeling
from utils import draw_detection

VIDEO_FILE = 'input/demo_images/volley 4.mp4'

def process_video(input_video_path):
    model = SceneLabeling()
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Process each frame with the scene annotation model
        frame_number += 1
        image_name = f"frame_{frame_number}.jpg"  # You can change this naming convention
        model.process_image(frame, image_name)
        annotation_results = model.get_image_annotations(image_name)

        print(f'Frame {frame_number} annotations:')
        for ar in annotation_results:
            if ("person" not in ar['spatial_relationship']) and ("Interacting" in ar['general_interaction']):
                print('Spatial relationship: ' + ar['spatial_relationship']
                      + '. General interaction: ' + ar['general_interaction'] + '. Person interaction: '
                      + ar['person_interaction'])
                arg_box = json.loads(ar['arg_bounding_box'])
                arg_label = ar['arg_label']
                ref_box = json.loads(ar['ref_bounding_box'])
                ref_label = ar['ref_label']
                draw_detection(frame.copy(), [arg_box, ref_box], [arg_label, ref_label])

            # Display the frame with annotations (you can save or display as needed)
                cv2.imshow('Video with Annotations', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    process_video(VIDEO_FILE)


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: ./input/models/yolo/yolov3.cfg in function 'cv::dnn::dnn4_v20230620::readNetFromDarknet'
